In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
places = pd.read_csv('Review_db.csv')

In [ ]:
places.shape

In [ ]:
places.head()

In [ ]:
places.duplicated().sum()

# Data Pre-Processing 

In [ ]:
places = places.drop_duplicates()

In [ ]:
places.drop(columns='Name', inplace=True)

In [ ]:
places.drop(columns='Date', inplace=True)

In [ ]:
places.duplicated(subset=['City', 'Place']).sum()

In [ ]:
places[places['City']=='Aamby Valley City']

In [ ]:
places['Rating'].max()

In [ ]:
places[places['City'] == 'Aamby Valley City']['Raw_Review'][0]

In [ ]:
places[places['City'] == 'Aamby Valley City']['Review'][0]

In [ ]:
places.drop(columns='Raw_Review', inplace=True)

In [ ]:
places

In [ ]:
new_places = pd.DataFrame()

In [ ]:
places[places['City'] == 'Kanpur']['Place'].unique()

In [ ]:
new_places[['City', 'Place']] = places[['City', 'Place']].drop_duplicates()

In [ ]:
new_places

In [ ]:
new_places.values

In [ ]:
places[places['City']=='Agra'][places['Place']=='Taj Mahal']['Rating'].count()

In [ ]:
new_places[new_places['City']=='Agra'][new_places['Place'] == 'Taj Mahal']['Review'].values[0]

In [ ]:
text = new_places[new_places['Place'] == 'Safed Akhara']['Review'].values[0]
text

In [ ]:
text1 = ''.join([i if ord(i) < 128 else '' for i in text])
text1

In [ ]:
re.sub(' +',' ',text1)

In [ ]:
rating = []
for val in new_places.values:
    rating.append(places[places['City']==val[0]][places['Place']==val[1]]['Rating'].mean())

In [ ]:
review = []
for val in new_places.values:
    combine = lambda x: "".join(x)
    review.append(combine(places[places['City']==val[0]][places['Place']==val[1]]['Review'].values))

In [ ]:
count = []
for val in new_places.values:
    count.append(places[places['City']==val[0]][places['Place']==val[1]]['Rating'].count())

In [ ]:
new_places['Review'] = review

In [ ]:
new_places['Rating'] = rating

In [ ]:
new_places['Count'] = count

<h2>Function to remove non ascii character from text</h2>

In [ ]:
def remove_non_ascii(text):
    text1 = ''.join([i if ord(i) < 128 else '' for i in text])
    return re.sub(' +',' ',text1)

In [ ]:
new_places['Review']=new_places['Review'].apply(lambda x: remove_non_ascii(x))

In [ ]:
new_places.to_csv('places.csv', header=True, index=True)

In [ ]:
new_places.shape

In [ ]:
new_places['Review'][0]

In [ ]:
# remove repetitive words
new_places['Review'] = new_places['Review'].apply(lambda x:' '.join(set(x.split())))

In [ ]:
new_places['Review'][0]

In [ ]:
total_review = ''
for str in df['Review'].values:
    total_review += str

In [ ]:
len(total_review)

In [ ]:
len(set(total_review.split()))

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [ ]:
new_places['Review'] = new_places['Review'].apply(stem)

In [ ]:
new_places['Review'][0]

<h2>Function to remove underscore from the string</h2>

In [ ]:
def remove_underscore(text):
    return text.replace('_', ' ')

In [ ]:
new_places['Review'] = new_places['Review'].apply(remove_underscore)

In [ ]:
new_places.to_csv('places.csv', header=True, index=False)

In [ ]:
df = pd.read_csv('places.csv')

In [ ]:
df.head()

In [ ]:
df[df['City']=='Agra'][df['Place']=='Taj Mahal']

In [ ]:
np.array(df['Review'].values)

# TF-IDF vectorization and cosine similarity

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')

In [ ]:
tfid_matrix = vectorizer.fit_transform(df['Review'])

In [ ]:
vectorizer.get_feature_names()

In [ ]:
tfid_matrix.shape

In [ ]:
tfid_matrix[0]

In [ ]:
cosine_similarity1 = linear_kernel(tfid_matrix[0:5000, :], tfid_matrix) 

In [ ]:
cosine_similarity2 = linear_kernel(tfid_matrix[5000:10000, :], tfid_matrix) 

In [ ]:
cosine_similarity3 = linear_kernel(tfid_matrix[10000:15239, :], tfid_matrix) 

In [ ]:
cosine_similarity = linear_kernel(tfid_matrix, tfid_matrix) 

In [ ]:
cosine_similarity3.shape

# Recommender Function

In [ ]:
def recommend(City, Place):
    index = df[df['City'] == City][df['Place']==Place].index[0]
    if(index<5000):
        distances = sorted(list(enumerate(cosine_similarity1[index])), reverse=True,key = lambda x: x[1])
    elif(index<10000):
        distances = sorted(list(enumerate(cosine_similarity2[5000-index])), reverse=True,key = lambda x: x[1])
    else:
        distances = sorted(list(enumerate(cosine_similarity3[10000-index])), reverse=True,key = lambda x: x[1])        
    for i in distances[1:6]:
        print(df.iloc[i[0]].City,", ", df.iloc[i[0]].Place)

In [ ]:
recommend('Agra', 'Taj Mahal')

# Downloading the pickle file

In [ ]:
import gzip, pickle, pickletools

In [ ]:
pickle.dump(cosine_similarity, open("cosine_similarity.pkl", "wb"))

In [ ]:
pickle.dump(cosine_similarity1, open("cosine_similarity1.pkl", "wb"))

In [ ]:
pickle.dump(cosine_similarity2, open("cosine_similarity2.pkl", "wb"))

In [ ]:
pickle.dump(cosine_similarity3, open("cosine_similarity3.pkl", "wb"))

In [ ]:
pickle.dump(df, open("travel.pkl", "wb"))

In [ ]:
def download_pickle(filepath, data):
    with gzip.open(filepath, "wb") as f:
        pickled = pickle.dumps(data)
        optimized_pickle = pickletools.optimize(pickled)
        f.write(optimized_pickle)

In [ ]:
download_pickle("cosine_similarity1.pkl", cosine_similarity1)

In [ ]:
download_pickle("cosine_similarity2.pkl", cosine_similarity2)

In [ ]:
download_pickle("cosine_similarity3.pkl", cosine_similarity3)

In [ ]:
download_pickle("travel.pkl", df)